# Hochschule Bonn-Rhein-Sieg

# Probabilistic Reasoning, WS21

# Assignment 03


Instructions for submission :
- Please restart and run all cells before submitting 
- Make sure your user name is correct
- No need to submit a pdf file, only the ipython is sufficient
- If you cannot code the solution , write down in words how you would go about solving it. 
- Write down theory questions in markdown format

Good luck !!


### Student user name: nmursa2s


In [1]:
from collections import defaultdict
import numpy

## Exercise 1: Robot - Diagnosis Using a Bayesian Network (40 points)

Let's consider the following Bayesian network, which is a simplified version of a robot diagnosis network:

![alt](figures/bot_diagnosis_net.png)

1) Assuming that the variable $Battery$ takes the values $\{$ empty, low, normal $\}$, $Laser$ the values $\{$ working, not working, constant nonzero measurement, zero measurement $\}$, while all the other variables take the values $\{$ working, not working $\}$:

* how many parameters are required for describing the probability distribution represented by the network?
* how many parameters do we need to describe the full joint distribution (without any independence assumptions)?

Justify your answers.

2) Let's now assume that we have added some components to our youBot base:

* an arm is attached (the youBot arm has five joints)
* a camera is connected to the base
* a second laser scanner is added as well
* a WiFi adapter is attached so that the robot can communicate with external components

Expand the existing network by including variables for the newly added components in it. Use the provided *figures/bot_diagnosis_net.dia* file for editing the network, export the network to a PNG image, and include your resulting network below.

\#\#\# Replace the image below with your resulting network
\#\#\#
![alt](figures/ex1.png)

Assuming that the newly added variables take the values $\{$ working, not working $\}$ - except for the variables that are similar to the existing non-binary ones:

* how many parameters are required for describing the probability distribution represented by *your* network?
* how many parameters do we need to describe the full joint distribution (without any independence assumptions)?

3) Your task now is to do one of the most challenging things when designing a Bayesian network - specifying the conditional probability tables of the variables in the network. Unless we have some data we can learn these from - and we don't have any data now - the probabilities have to be specified manually, an activity that requires quite a lot of expertise and domain knowledge. For the sake of this problem, you thus have to imagine that you are indeed a youBot expert.

Write the conditional probability tables of *your* diagnosis network, namely the one that you created in part (b). Justify why it makes sense to assign the probabilities just as you have done.


### 1.a
In this case we need to multiply the values of each variables and get the result. Thus, 
$$RESULT = 4*2+6+2+3=19$$

### 1.b
As we know, a joint distribution for a network with n values Boolean nodes has $n^2-1$ rows for the combinations of parent values. Thus, $$RESULT=(3*2*2*2*2*2*4)-1=383$$


### 2.a
As we have new values each of them with 2 parameters we need to multiply the previous result we found with these values. So, $$RESULT=192 \times 2 \times 2 \times 2 \times 2=3072$$

### 2.b
We will have $7+4$ variables in this case, so $$2ˆ11-1=2028-1=2027$$

### 3
![alt](figures/ex13.jpg)

## Exercise 2: Using inferencing tools (30 Points)


Given below is a sample code for inferencing of a sample joint probability case. An example for how it is used in the case of tooth and cavity is also given. Use this example to understand how the code works.

1) Now use this code to perform 3 types of inferences of your choosing for the Bayes network(the conditional probability tables values) you defined in Exercise 1 part 3

In [2]:
class utils:
    
    def extend(s, var, val):
        """Copy dict s and extend it by setting var to val; return copy."""
        
        return {**s, var: val}
    
class ProbDist:
    """A discrete probability distribution. You name the random variable
    in the constructor, then assign and query probability of values.
    >>> P = ProbDist('Flip'); P['H'], P['T'] = 0.25, 0.75; P['H']
    0.25
    >>> P = ProbDist('X', {'lo': 125, 'med': 375, 'hi': 500})
    >>> P['lo'], P['med'], P['hi']
    (0.125, 0.375, 0.5)
    """

    def __init__(self, varname='?', freqs=None):
        """If freqs is given, it is a dictionary of values - frequency pairs,
        then ProbDist is normalized."""
        self.prob = {}
        self.varname = varname
        self.values = []
        if freqs:
            for (v, p) in freqs.items():
                self[v] = p
            self.normalize()

    def __getitem__(self, val):
        """Given a value, return P(value)."""
        try:
            return self.prob[val]
        except KeyError:
            return 0

    def __setitem__(self, val, p):
        """Set P(val) = p."""
        if val not in self.values:
            self.values.append(val)
        self.prob[val] = p

    def normalize(self):
        """Make sure the probabilities of all values sum to 1.
        Returns the normalized distribution.
        Raises a ZeroDivisionError if the sum of the values is 0."""
        total = sum(self.prob.values())
        if not isclose(total, 1.0):
            for val in self.prob:
                self.prob[val] /= total
        return self

    def show_approx(self, numfmt='{:.3g}'):
        """Show the probabilities rounded and sorted by key, for the
        sake of portable doctests."""
        return ', '.join([('{}: ' + numfmt).format(v, p)
                          for (v, p) in sorted(self.prob.items())])

    def __repr__(self):
        return "P({})".format(self.varname)

    
    
class JointProbDist(ProbDist):
    """A discrete probability distribute over a set of variables.
      """

    def __init__(self, variables):
        self.prob = {}
        self.variables = variables
        self.vals = defaultdict(list)

    def __getitem__(self, values):
        """Given a tuple or dict of values, return P(values)."""
        values = event_values(values, self.variables)
        return ProbDist.__getitem__(self, values)

    def __setitem__(self, values, p):
        """Set P(values) = p. Values can be a tuple or a dict; it must
        have a value for each of the variables in the joint. Also keep track
        of the values we have seen so far for each variable."""
        values = event_values(values, self.variables)
        self.prob[values] = p
        for var, val in zip(self.variables, values):
            if val not in self.vals[var]:
                self.vals[var].append(val)

    def values(self, var):
        """Return the set of possible values for a variable."""
        return self.vals[var]   
   

    def __repr__(self):
        return "P({})".format(self.variables)
    
    
    

##Set of utility functions

def extend(s, var, val):
    """Copy dict s and extend it by setting var to val; return copy."""

    return {**s, var: val}


def event_values(event, variables):
    """Return a tuple of the values of variables in event.
    >>> event_values ({'A': 10, 'B': 9, 'C': 8}, ['C', 'A'])
    (8, 10)
    >>> event_values ((1, 2), ['C', 'A'])
    (1, 2)
    """
    if isinstance(event, tuple) and len(event) == len(variables):
        return event
    else:
        return tuple([event[var] for var in variables])

def enumerate_joint_ask(X, e, P):
    """
    
    Return a probability distribution over the values of the variable X,
    given the {var:val} observations e, in the JointProbDist P.
  
    """
    assert X not in e, "Query variable must be distinct from evidence"
    Q = ProbDist(X)  # probability distribution for X, initially empty
    Y = [v for v in P.variables if v != X and v not in e]  # hidden variables.
    for xi in P.values(X):
        Q[xi] = enumerate_joint(Y, extend(e, X, xi), P)
    return Q.normalize()


def enumerate_joint(variables, e, P):
    """Return the sum of those entries in P consistent with e,
    provided variables is P's remaining variables (the ones not in e)."""
    if not variables:
        return P[e]
    Y, rest = variables[0], variables[1:]
    return sum([enumerate_joint(rest, extend(e, Y, y), P) for y in P.values(Y)])




In [3]:
##Defing the random variables involved
full_joint = JointProbDist(['Cavity', 'Toothache', 'Catch'])
## Definging the full joint distribution
full_joint[dict(Cavity=True, Toothache=True, Catch=True)] = 0.108
full_joint[dict(Cavity=True, Toothache=True, Catch=False)] = 0.012
full_joint[dict(Cavity=True, Toothache=False, Catch=True)] = 0.016
full_joint[dict(Cavity=True, Toothache=False, Catch=False)] = 0.064
full_joint[dict(Cavity=False, Toothache=True, Catch=True)] = 0.072
full_joint[dict(Cavity=False, Toothache=False, Catch=True)] = 0.144
full_joint[dict(Cavity=False, Toothache=True, Catch=False)] = 0.008
full_joint[dict(Cavity=False, Toothache=False, Catch=False)] = 0.576

In [4]:
# IN OUR CASE WE WILL HAVE SOMETHING LIKE THIS 
full_joint = JointProbDist(['Camera','Battery','Base', 'Laser1', 'Laser2','Arm', 'Joint1','Joint2', 'Joint3', 'Joint4'])

full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=True, Laser2=True, Arm=True, Joint1=True, Joint2=True, Joint3=True, Joint4=True)] = 0.009
full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=True, Laser2=True, Arm=True, Joint1=True, Joint2=True, Joint3=True, Joint4=False)] = 0.015
full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=True, Laser2=True, Arm=True, Joint1=True, Joint2=True, Joint3=False, Joint4=True)] = 0.023
full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=True, Laser2=True, Arm=True, Joint1=False, Joint2=True, Joint3=True, Joint4=True)] = 0.011
full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=True, Laser2=True, Arm=False, Joint1=True, Joint2=True, Joint3=True, Joint4=True)] = 0.027
full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=True, Laser2=False, Arm=True, Joint1=True, Joint2=True, Joint3=True, Joint4=True)] = 0.031
full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=False, Laser2=True, Arm=True, Joint1=True, Joint2=True, Joint3=True, Joint4=True)] = 0.004
full_joint[dict(Camera=True, Battery=True, Base=False, Laser1=True, Laser2=True, Arm=True, Joint1=True, Joint2=True, Joint3=True, Joint4=True)] = 0.0012
full_joint[dict(Camera=True, Battery=False, Base=True, Laser1=True, Laser2=True, Arm=True, Joint1=True, Joint2=True, Joint3=True, Joint4=True)] = 0.010
full_joint[dict(Camera=False, Battery=True, Base=True, Laser1=True, Laser2=True, Arm=True, Joint1=True, Joint2=True, Joint3=True, Joint4=True)] = 0.027
full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=True, Laser2=True, Arm=True, Joint1=True, Joint2=False, Joint3=True, Joint4=False)] = 0.017
full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=True, Laser2=True, Arm=True, Joint1=False, Joint2=False,Joint3=True, Joint4=False)] = 0.007
full_joint[dict(Camera=True, Battery=True, Base=True, Laser1=True, Laser2=True, Arm=False, Joint1=False, Joint2=False, Joint3=True,  Joint4=False)] = 0.018


In [5]:
## Performing inferencing
evidence = dict(Camera=True, Battery=False, Base=True, Laser1=False, Laser2=True, Arm=False, Joint1=True, Joint2=True, Joint3=True, Joint4=True)
variables = ['Laser1']# The hidden variables that are not part of but required for the marginalizatio
ans = enumerate_joint(variables, evidence, full_joint)
print(f'{ans}')

0


## Exercise 3:  Bayesian Network (30 points) 
*Source: Russel, Norvig: "Artificial Intelligence, a modern Approach"*

The image below shows a bayesian network describing some features of a car’s electrical system
and engine. Each variable is Boolean, and the true value indicates that the corresponding
aspect of the vehicle is in working order.

![alt](figures/ex3.png)


1.) Extend the network with the Boolean variables *IcyWeather* and *StarterMotor*. <br>
2.) Give reasonable conditional probability tables for all the nodes.<br>
3.) How many independent values are contained in the joint probability distribution for eight
Boolean nodes, assuming that no conditional independence relations are known to hold
among them?<br>
4.) How many independent probability values do your network tables contain?<br>
5.) The conditional distribution for *Starts* could be described as a noisy-AND distribution.
Define this family in general and relate it to the noisy-OR distribution.

### Answer
1.  ![alt](figures/ex3_solution.png) 

2. It is up to us to choose the conditional probabilities for all the nodes, so we might come up with this following values:
        P(IcyWeather) = 0.5 

        P(Battery|IcyWeather)  = 0.95; P(Battery|~IcyWeather)  = 0.997 

        P(SMW|IcyWeather)  = 0.98; P(SMW|~IcyWeather)  = 0.999

        P(Radio|Battery)  = 0.999; P(Radio|~Battery)  = 0.05

        P(Ignition|Battery)  = 0.998; P(Ignition|~Battery)  = 0.01

        P(Gas) = 0.995

        P(Starts|Ignition, SMW, Gas) = 0.9999

        All other entries are zero

        P(Moves|Starts) = 0.998

3. There are 8 Boolean variables here. The joint has $2ˆ8–1=255$ independent entries.
4. With our new figure, we can see that our IcyWeather has 1 entry, gas has 1 entry, starts has 8 entries and others has 2 entries. Thus, in this case, the result will be $1 + 1 + 3 + 8 + 2 + 2 +2=20$
5. As we know, Starts has 8 entries. These entries describe a set of necessary condition for the motor to start. As w
seen from our figure, we can see that the engine starts only if all three antrecedents are satisfied. So except for that entry, all others will be zero. The entry for which the engine starts is fairly close to 1, not quite.  As learn more about the problem, we can add more and more conditions and this entry will move closer and closer to 1.